<a href="https://colab.research.google.com/github/allisonlinn/CSUREMM/blob/main/Modeling_1st_Try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install pandas
!pip install google-colab
!pip install sklearn
!pip install statsmodels

from google.colab import files
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from statsmodels.tsa.stattools import grangercausalitytests

  Using cached sklearn-0.0.post5.tar.gz (3.7 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [27]:
# Upload CSV file from local machine
uploaded = files.upload()

# Get the file name
file_name = next(iter(uploaded))

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_name)

Saving 7_6_23 - Vader + Volatility - Final (3).csv to 7_6_23 - Vader + Volatility - Final (3).csv


In [44]:
data_filtered = df.dropna(subset=['VUIAX_P'])

# Extract the 'compound' and 'VCDAX' columns
x = data_filtered['compound'].values.reshape(-1, 1)
y = data_filtered['VUIAX_P'].values

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Initialize a dictionary to store the R-squared values for each method
r_squared_values = {}

# Ordinary Least Squares (OLS)
model_ols = LinearRegression()
model_ols.fit(x_train, y_train)
y_pred_ols = model_ols.predict(x_test)
r_squared_values['OLS'] = r2_score(y_test, y_pred_ols)

# Linear Regression
model_linear = LinearRegression()
model_linear.fit(x_train, y_train)
y_pred_linear = model_linear.predict(x_test)
r_squared_values['Linear Regression'] = r2_score(y_test, y_pred_linear)

# Polynomial Regression (degree = 2)
poly_features = PolynomialFeatures(degree=2)
x_poly = poly_features.fit_transform(x_train)
model_poly = LinearRegression()
model_poly.fit(x_poly, y_train)
x_poly_test = poly_features.transform(x_test)
y_pred_poly = model_poly.predict(x_poly_test)
r_squared_values['Polynomial Regression (Degree 2)'] = r2_score(y_test, y_pred_poly)

# Polynomial Regression (degree = 3)
poly_features = PolynomialFeatures(degree=3)
x_poly = poly_features.fit_transform(x_train)
model_poly = LinearRegression()
model_poly.fit(x_poly, y_train)
x_poly_test = poly_features.transform(x_test)
y_pred_poly = model_poly.predict(x_poly_test)
r_squared_values['Polynomial Regression (Degree 3)'] = r2_score(y_test, y_pred_poly)

# Print the R-squared values for each method
for method, r_squared in r_squared_values.items():
    print(f"{method}: R-squared = {r_squared:.4f}")

OLS: R-squared = 0.0070
Linear Regression: R-squared = 0.0070
Polynomial Regression (Degree 2): R-squared = 0.0689
Polynomial Regression (Degree 3): R-squared = 0.0722


In [51]:
# Drop rows with NaN values in 'compound' and the target columns
columns =    ["compound","VCDAX_V", "VCSAX_V", "VENAX_V", "VFAIX_V", "VGSLX_V", "VINAX_V", "VITAX_V", "VMAIX_V", "VSPVX_V", "VTCAX_V",
           "VUIAX_V", "VINAX_V", "VUIAX_P", "VGHCX_P", "VFAIX_P", "VSPVX_P", "VITAX_P", "VMIAX_P", "VGSLX_P", "VINAX_P",
           "VTCAX_P", "VCSAX_P", "VENAX_P", "VCDAX_P"]

data_filtered = df[columns].dropna()

# Perform the Granger causality test for each target column
max_lag = 5  # Maximum lag to consider

for target_column in columns[1:]:
    print(f"Testing Granger causality: compound -> {target_column}")
    granger_data = data_filtered[[target_column, 'compound']]

    # Reshape the data to have a 1-dimensional shape
    granger_data = granger_data.values.reshape(-1, 2)

    results = grangercausalitytests(granger_data, max_lag, verbose=False)

    # Extract the p-values from the test results
    p_values = [results[i + 1][0]['ssr_ftest'][1] for i in range(max_lag)]

    # Print the p-values and interpretation
    for lag, p_value in enumerate(p_values):
        print(f"Lag {lag + 1}: p-value = {p_value:.4f}")
        if p_value < 0.05:
            print("  Granger causality is statistically significant.")
        else:
            print("  Granger causality is not statistically significant.")
    print()


Testing Granger causality: compound -> VCDAX_V
Lag 1: p-value = 0.5363
  Granger causality is not statistically significant.
Lag 2: p-value = 0.6941
  Granger causality is not statistically significant.
Lag 3: p-value = 0.5375
  Granger causality is not statistically significant.
Lag 4: p-value = 0.6585
  Granger causality is not statistically significant.
Lag 5: p-value = 0.4042
  Granger causality is not statistically significant.

Testing Granger causality: compound -> VCSAX_V
Lag 1: p-value = 0.0375
  Granger causality is statistically significant.
Lag 2: p-value = 0.1147
  Granger causality is not statistically significant.
Lag 3: p-value = 0.0015
  Granger causality is statistically significant.
Lag 4: p-value = 0.0032
  Granger causality is statistically significant.
Lag 5: p-value = 0.0067
  Granger causality is statistically significant.

Testing Granger causality: compound -> VENAX_V
Lag 1: p-value = 0.3422
  Granger causality is not statistically significant.
Lag 2: p-value 